In [1]:
import numpy as np
import cv2
import os
import import_ipynb
import qrcode
import uuid
import random

QRcode_size = 31

In [2]:
def genQRcode(id = 0):
    string = str(uuid.uuid4())
    # 35x35 QRcode
    img = qrcode.make(string, box_size=1, border=1)

    img.save('QRcode.png')
    img = cv2.imread('QRcode.png')
    img = cv2.resize(img, (QRcode_size, QRcode_size))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype(np.uint8)
    os.remove('QRcode.png')
    return img

In [3]:
def LSBQRcode(target_img_path: str, QRcode: np.ndarray, if_random: bool = False, which_channel: int = 0) -> np.ndarray:
    img = cv2.imread(target_img_path)

    # get blue channel
    img_b = img[:,:,0]
    # get green channel
    img_g = img[:,:,1]
    # get red channel
    img_r = img[:,:,2]

    if which_channel == 0:
        select_channel = img_b
    elif which_channel == 1:
        select_channel = img_g
    elif which_channel == 2:
        select_channel = img_r

    select_channel &= 0b11111110

    start_point = [0, 0]
    if if_random:
        start_point = [random.randint(0, img.shape[0] - QRcode_size - 1), random.randint(0, img.shape[1] - QRcode_size - 1)]

    for i in range(QRcode.shape[0]):
        for j in range(QRcode.shape[1]):
            if QRcode[i][j] == 255:
                select_channel[start_point[0] + i][start_point[1] + j] |= 0b00000001
    
    img[:,:,which_channel] = select_channel
    
    return img

In [4]:
def LSBQRcodeDecode(target_img_path:str):
    img = cv2.imread(target_img_path)

    # get blue channel
    img_b = img[:,:,0]
    # get green channel
    img_g = img[:,:,1]
    # get red channel
    img_r = img[:,:,2]

    img_b = img_b & 0b00000001

    img_b = img_b * 255

    # check if the QRcode is in the image
    if np.sum(img_b) == 0:
        return False
    else:
        return True


In [5]:
from tqdm import tqdm
# get input images
img_path = "../imageDatabase/outputImages/96/"
img_list = os.listdir(img_path)

index = 0

progress_bar = tqdm(total=len(img_list), position=0, leave=True)
for img_name in img_list:
    progress_bar.update(1)
    # png or jpg
    if img_name[-3:] == "png" or img_name[-3:] == "jpg":
        img = cv2.imread(img_path + img_name)
        # check size if > 35x35
        if img.shape[0] > QRcode_size and img.shape[1] > QRcode_size:
            QRcode = genQRcode(index)
            img = LSBQRcode(img_path + img_name, QRcode, if_random=True, which_channel=0)
            cv2.imwrite("./outputImages/" + "_" + img_name, img)
            # print("Image " + img_name + " done")
            # print("Check if QRcode is in the image: " + str(LSBQRcodeDecode("./outputImages/" + img_name)))
            index += 1
        # else:
             # print("Image size is too small")

progress_bar.close()



100%|████████████████████████████████████████████████████████████████████████████| 10200/10200 [04:17<00:00, 39.65it/s]
